In [1]:
import numpy as np
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
from Models.selective_sequential import *
from Loss.triplet_regularized import *
from session import *
from LR_Schedule.cyclical import Cyclical
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *
import util
from session import LossMeter, EvalModel
from Layers.flatten import Flatten
from torch.utils.tensorboard import SummaryWriter

In [2]:
%load_ext autoreload
%autoreload 2

torch.cuda.set_device(0); torch.backends.cudnn.benchmark=True;

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/torch/cuda/__init__.py:135: UserWarning: 
    Found GPU0 GeForce GTX 770 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST('/media/drake/MX500/Datasets/mnist/train', download=True, train=True, transform=transform)
partial_trainset = torch.utils.data.dataset.Subset(trainset, np.arange(500))

valset = datasets.MNIST('/media/drake/MX500/Datasets/mnist/test', download=True, train=False, transform=transform)
partial_valset = torch.utils.data.dataset.Subset(valset, np.arange(500))

trainloader = torch.utils.data.DataLoader(partial_trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(partial_valset, batch_size=64, shuffle=False)

In [4]:
class Identity(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
    def forward(self, x):
        return x

select = ['act1', 'act2', 'out']

def make_model(dropout=False, batchnorm=False):
    return SelectiveSequential(
    select,
    {'conv64': nn.Conv2d(1, 64, kernel_size=5, padding=2),
     'bn64': nn.BatchNorm2d(num_features=64) if batchnorm else Identity(),
     'act64': nn.ReLU(True),

     'max1': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-64': nn.Dropout(.05) if dropout else Identity(),  
    
     'conv192': nn.Conv2d(64, 192, kernel_size=5, padding=2),
     'bn192': nn.BatchNorm2d(num_features=192) if batchnorm else Identity(),
     'act192': nn.ReLU(True),   

     'max2': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-192': nn.Dropout(.1) if dropout else Identity(),
    
     'conv384': nn.Conv2d(192, 384, kernel_size=3, padding=1),
     'bn384': nn.BatchNorm2d(num_features=384) if batchnorm else Identity(),
     'act384': nn.ReLU(True),
     'drop-384': nn.Dropout(.15) if dropout else Identity(),
     
     'conv256a': nn.Conv2d(384, 256, kernel_size=3, padding=1),
     'bn256a': nn.BatchNorm2d(num_features=256) if batchnorm else Identity(),
     'act256a': nn.ReLU(True),
     'drop-256a': nn.Dropout(.1) if dropout else Identity(),
     
     'conv256b': nn.Conv2d(256, 256, kernel_size=3, padding=1),
     'bn256b': nn.BatchNorm2d(num_features=256) if batchnorm else Identity(),
     'act256b': nn.ReLU(True),

     'max3': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-256b': nn.Dropout(.1) if dropout else Identity(),
    
     'flatten': Flatten(),

     'fc1': nn.Linear(3 * 3 * 256, 512),
     'act1': nn.ReLU(True),
     'drop-fc1': nn.Dropout(.05) if dropout else Identity(),

     'fc2': nn.Linear(512, 512),
     'act2': nn.ReLU(True),
     'drop-fc2': nn.Dropout(.05) if dropout else Identity(),

     'out': nn.Linear(512, 10)})

In [7]:
accuracies = {}

for param in [0, .1, .2, .3, .4, .5, .6, .7]:
    criterion = TripletRegularizedMultiMarginLoss(param, .5, select)
    sess = Session(make_model(False, True), criterion, optim.AdamW, 1e-4)
    num_epochs = 50
    validator = EmbeddingSpaceValidator(valloader, select, CustomOneHotAccuracy, tensorboard_dir=f"./runs/lambda{param}")
    lr_scheduler = CosAnneal(len(trainloader)*num_epochs, T_mult=1, lr_min=1e-6)
    schedule = TrainingSchedule(trainloader, num_epochs, [lr_scheduler, validator])
    sess.train(schedule)
    
    accuracies[param] = validator.best_accuracy
    df = pd.DataFrame(accuracies)
    df.to_csv("./MNIST-SEARCH.csv")


val accuracy:  0.134 train accuracy:  0.264 
train loss:  1.1676  train unreg loss :  0.7411 
valid loss:  1.253  valid unreg loss :  0.8916



val accuracy:  0.148 train accuracy:  0.798 
train loss:  0.9888  train unreg loss :  0.4166 
valid loss:  1.0814  valid unreg loss :  0.845



val accuracy:  0.434 train accuracy:  0.912 
train loss:  0.7965  train unreg loss :  0.2389 
valid loss:  0.9273  valid unreg loss :  0.6837



val accuracy:  0.842 train accuracy:  0.994 
train loss:  0.6368  train unreg loss :  0.132 
valid loss:  0.8306  valid unreg loss :  0.4661



val accuracy:  0.838 train accuracy:  1.0 
train loss:  0.5511  train unreg loss :  0.094 
valid loss:  0.7714  valid unreg loss :  0.2869


...


val accuracy:  0.95 train accuracy:  1.0 
train loss:  0.0121  train unreg loss :  0.0008 
valid loss:  1.6111  valid unreg loss :  0.056



val accuracy:  0.952 train accuracy:  1.0 
train loss:  0.0131  train unreg loss :  0.0008 
valid loss:  1.6132  valid unreg loss :  0.0558



val accuracy:  0.95 train accuracy:  1.0 
train loss:  0.0113  train unreg loss :  0.0008 
valid loss:  1.611  valid unreg loss :  0.0558



val accuracy:  0.952 train accuracy:  1.0 
train loss:  0.0097  train unreg loss :  0.0007 
valid loss:  1.6121  valid unreg loss :  0.0559



val accuracy:  0.95 train accuracy:  1.0 
train loss:  0.0084  train unreg loss :  0.0007 
valid loss:  1.6114  valid unreg loss :  0.0559

